# 빅데이터 분석기사 4회 실기 기출 유형
---
- [마케팅] 자동차 시장 세분화
자동차 회사는 새로운 전략을 수립하기 위해 4개의 시장으로 세분화했습니다.
기존 고객 분류 자료를 바탕으로 신규 고객이 어떤 분류에 속할지 예측해주세요!
예측할 값(y): "Segmentation" (1,2,3,4)
평가: Macro f1-score
data: train.csv, test.csv

- 제출 형식:  
ID,Segmentation  
458989,1  
458994,2   
459000,3  
459003,4  
  
  
- 답안 제출 참고    
아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용  
pd.DataFrame({'ID': test.ID, 'Segmentation': pred}).to_csv('003000000.csv', index=False)  
  
  
- 튜토리얼 노트북  
링크: https://www.kaggle.com/code/agileteam/4th-t2-python   
basic: 수치형 데이터만 활용 -> 학습 및 test데이터 예측  
intermediate: 범주형 데이터도 활용 -> 학습 및 test데이터 예측   
advanced: 학습 및 교차 검증(모델 평가) -> 하이퍼파라미터 튜닝 -> test데이터 예측  
  
- 학습을 위한 채점  
최종 파일을 "수험번호.csv"가 아닌 "submission.csv" 작성 후 오른쪽 메뉴 아래 "submit" 버튼 클릭 -> 리더보드에 점수 및 등수 확인 가능함  
pd.DataFrame({'ID': test.ID, 'Segmentation': pred}).to_csv('submission.csv', index=False)  

# 모듈로딩 & 데이터 불러오기
---

In [87]:
import pandas as pd
import numpy as np
train = pd.read_csv('./Segmentation(t2-4회)/train.csv')
test = pd.read_csv('./Segmentation(t2-4회)/test.csv')

In [88]:
# 제출 양식 ID, Segmentation
ID = test['ID']

# EDA
---

In [89]:
train

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,4
1,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,2
2,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,2
3,461319,Male,Yes,56,No,Artist,0.0,Average,2.0,Cat_6,3
4,460156,Male,No,32,Yes,Healthcare,1.0,Low,3.0,Cat_6,3
...,...,...,...,...,...,...,...,...,...,...,...
6660,463002,Male,Yes,41,Yes,Artist,0.0,High,5.0,Cat_6,2
6661,464685,Male,No,35,No,Executive,3.0,Low,4.0,Cat_4,4
6662,465406,Female,No,33,Yes,Healthcare,1.0,Low,1.0,Cat_6,4
6663,467299,Female,No,27,Yes,Healthcare,1.0,Low,4.0,Cat_6,2


In [90]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               6665 non-null   int64  
 1   Gender           6665 non-null   object 
 2   Ever_Married     6665 non-null   object 
 3   Age              6665 non-null   int64  
 4   Graduated        6665 non-null   object 
 5   Profession       6665 non-null   object 
 6   Work_Experience  6665 non-null   float64
 7   Spending_Score   6665 non-null   object 
 8   Family_Size      6665 non-null   float64
 9   Var_1            6665 non-null   object 
 10  Segmentation     6665 non-null   int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 572.9+ KB


In [91]:
train.corr()

C:\Users\USER\AppData\Local\Temp\ipykernel_2800\2189804198.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train.corr()


,ID,Age,Work_Experience,Family_Size,Segmentation
ID,1.000000,0.011664,-0.028948,0.006343,-0.013782
Age,0.011664,1.000000,-0.188769,-0.281772,-0.231696
Work_Experience,-0.028948,-0.188769,1.000000,-0.069123,0.006982
Family_Size,0.006343,-0.281772,-0.069123,1.000000,0.199412
Segmentation,-0.013782,-0.231696,0.006982,0.199412,1.000000


In [92]:
# 범주형 ,수치형 나누기
train_cat = train.select_dtypes(['object'])
train_num = train.select_dtypes(['int'])

test_cat = test.select_dtypes(['object'])
test_num = test.select_dtypes(['int'])

In [93]:
train_cat.columns

Index(['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score',
       'Var_1'],
      dtype='object')

In [94]:
# 원핫
train_cat = pd.get_dummies(train_cat[['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score','Var_1']])
test_cat = pd.get_dummies(test_cat[['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score','Var_1']])


train = pd.concat([train_cat,train_num],axis=1)
test = pd.concat([test_cat,test_num],axis=1)
train

,Gender_Female,Gender_Male,Ever_Married_No,Ever_Married_Yes,Graduated_No,Graduated_Yes,Profession_Artist,Profession_Doctor,Profession_Engineer,Profession_Entertainment,...,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7,ID,Age,Segmentation
0,0,1,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,462809,22,4
1,1,0,0,1,0,1,0,0,1,0,...,0,0,0,0,0,1,0,466315,67,2
2,0,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,1,0,461735,67,2
3,0,1,0,1,1,0,1,0,0,0,...,0,0,0,0,0,1,0,461319,56,3
4,0,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,460156,32,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6660,0,1,0,1,0,1,1,0,0,0,...,0,0,0,0,0,1,0,463002,41,2
6661,0,1,1,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,464685,35,4
6662,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,465406,33,4
6663,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,467299,27,2


# 전처리
---

In [95]:
# train, target split
target = train['Segmentation']

train = train.drop(['Segmentation','ID'],axis=1)
test = test.drop(['Segmentation','ID'],axis=1)


# X_train,X_test,y_train,y_test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train,target,stratify =target,random_state=1)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

KeyError: "['Segmentation'] not found in axis"

In [ ]:
t